In [1]:
#GOt FROM
#https://github.com/hcmidt/corehd/blob/master/chouffe.py

## suboptimal, but fast, implementation of the corehd and weak-neighbor algorithm ##
## The code can be implemented by hirarchical binning, eading to better time complexity ##

import sys
import networkx as nx
from random import *
from operator import itemgetter
import matplotlib.pyplot as plt
def score(v,G,scm='HD'):
    # compute the score of node v in G

    if scm == 'HD':
    # high degree score : uniform
        scr = 0
    elif scm == 'WN':
    # weak-neighbor1 score
        scr = sum( (G.degree(nb) for nb in G[v]) )
        scr = - scr
    elif scm == 'SN':
    # weak-neighbor2 score
        if G.degree(v) != 0:
            scr = sum( (G.degree(nb) for nb in G[v]) )
            scr = G.degree(v) - scr/float(G.degree(v))
        else:
            scr = 0
    else:
        sys.exit("Error : score function is not defined.")

    return scr

def category(v,k,G,scm='HD'):
    # computes the right category to order node to
    if scm == 'HD':
        dgr = max(k-1,G.degree(v))
    elif scm == 'WN':
        dgr = max(k-1,G.degree(v))
    elif scm == 'SN':
        if G.degree(v) < k:
            dgr = k-1
        else:
            dgr = k
    else:
        sys.exit("Error : algorithm is not defined.")

    return dgr

def max_cat(k,G,scm='HD'):
    # computes the max category in dict
    if scm == 'HD':
        out = max(dict(G.degree()).values())
    elif scm == 'WN':
        out = max(G.degree().values())
    elif scm == 'SN':
        out = k
    else:
        sys.exit("Error : algorithm is not defined. Cannot obtain max.")

    return out

def preprocess(k,G,scm='HD'):
    # suboptimal, but O(|G|), computation of some initial properties

    # size of initial graph
    N0 = len(G)
    # compute the k-core in O(|G|) steps (not really necessary)
    G = nx.k_core(G,k)
    # size of the initial k-core
    N = len(G)

    # if the core is empty we are done
    if N == 0:
        return G, 0, dict(), dict(), dict(), N0, N

    dmax = max_cat(k,G,scm)

    # Initialize the dictionary, H, with 
    # H[degree][score] = {i_1: 1, ..., i_r:1} and i_j indicating a node
    # degree = k-1,...,d because it is not necessary to distinguish nodes of degree smaller k.
    # the k-1 nodes need not be organized by score, but it makes the code less prune to errors and doesn't cost us much
    H = { d: dict() for d in range(k-1,dmax+1) }

    # collect the score for every node 
    score_dict = {}
    # collect the max score for each 'degree' = k-1
    max_score_dict = {}

    for v in G.nodes():
        dgr = category(v,k,G,scm)
        scr = score(v,G,scm)
        score_dict[v] = scr
        # sort them in dictionaries by scores.
        try:
            H[dgr][scr][v] = 1
        except KeyError:
            # create if it does not already exist
            H[dgr][scr] = dict()
            H[dgr][scr][v] = 1

    # track currently largest score for each H[.] 
    for sub_dict_name in H.keys():
        try:
            mx_scr = max(H[sub_dict_name].keys())
        except ValueError:
            mx_scr = None 

        max_score_dict[sub_dict_name] = mx_scr


    return G, dmax, H, max_score_dict, score_dict, N0, N

#### JUST ADAPT d \in {<k or >k} for SN
def destroy(k,G,scm='HD'):
    # implementation of the generalized corehd algorithm (scm='HD')
    # and one version of the weak-neighbor algorithm (scm='WN')
    # we assume that G has no self-loops.
    G_copy = G.copy()
    ### initialize 
    # pre-processing
    G, dmax, H, max_score_dict, score_dict, N0, N = preprocess(k,G,scm)
    # set of seeds
    D = []
    if N == 0:
        while len(G_copy.nodes) != 0:
            index, element = max(G_copy.degree(), key=itemgetter(1))
            G_copy.remove_node(index)
            D.append(index)
        return D, _,_ 
    # current indicator for either removal (d=dmax) or trimming (d=k-1)
    if max_score_dict[k-1] == None:
        d = dmax
    else:
        d = k-1
    removed_node = []
    ### remove and trimm until the k-core is empty
    cnt = 0 ; done = False ;
    while cnt < N and done == False:
        cnt += 1
        ## remove node 
        # get the max score of degree d nodes
        mx_scr_d = max_score_dict[d]

        # remove one of them at random
        v = choice(list(H[d][mx_scr_d].keys()))

        # if d > k-1 add them to the seed set
        if d >= k:
            D.append(v)

        # remove element from H and set its score to None
        del H[d][mx_scr_d][v]
        score_dict[v] = None

        # check for newest largest score
        if H[d][mx_scr_d] == {}:
            del H[d][mx_scr_d]
        # update max_score_dict
        # suboptimal
        try:
            max_score_dict[d] = max(H[d].keys())
        except ValueError:
            max_score_dict[d] = None

        ## update the neighbors 
        dv = G[v]
        ddv = set()
        # remove neighbors from dict (for now)
        # other cases we will take care of below. E.g. updating max_score of the new degree
        for nb in dv:
            remove_node_by_score(nb,G,H,max_score_dict,score_dict,k,scm)
            ddv = ddv | set(G[nb].keys())

        # remove also neighbors neighbors (except v) because their score will change
        ddv = ddv - set([v])
        ddv = ddv - set(dv)
        for nnb in ddv:
            remove_node_by_score(nnb,G,H,max_score_dict,score_dict,k,scm)
        removed_node.append(v)
        ## remove v from G
        G.remove_node(v)

        # update the degrees of the neighbors and their scores
        for nb in dv:
            add_node_by_score(nb,G,H,max_score_dict,score_dict,k,scm)
        # update scores of the neighbors neighbors
        for nnb in ddv:
            add_node_by_score(nnb,G,H,max_score_dict,score_dict,k,scm)

        ## check if dmax needs updating and do so if necessary. not necessary
        if max_score_dict[d] == None:
            # attempt to update
            try: 
                while max_score_dict[dmax] == None:
                    dmax -= 1
            # unless there are no nodes left
            except KeyError:
                done = True
                #sys.exit('Error!')

        ## update d
        if max_score_dict[k-1] != None:
            d = k-1
        else:
            d = dmax
    #not_D =  [x for x in GG.nodes if x not in D]
    #not_D = list(G_copy.nodes)- list(D)
    G_copy.remove_nodes_from(D)
    while len(G_copy.nodes) != 0:
        index, element = max(G_copy.degree(), key=itemgetter(1))
        G_copy.remove_node(index)
        D.append(index)
    return D, N, removed_node 


def add_node_by_score(v,G,H,max_score_dict,score_dict,k,scm):
    # get new score
    score_dict[v] = score(v,G,scm)
    # get new category (degree) for dict reordering
    dgr = category(v,k,G,scm)
    # insert into proper spot
    try:
        H[dgr][score_dict[v]][v] = 1
    except KeyError:
        # create if it does not already exist
        H[dgr][score_dict[v]] = dict()
        H[dgr][score_dict[v]][v] = 1
    # update max_score
    if max_score_dict[dgr] == None or max_score_dict[dgr] < score_dict[v]:
        max_score_dict[dgr] = score_dict[v]

def remove_node_by_score(v,G,H,max_score_dict,score_dict,k,scm):
    # find the right dict
    dgr = category(v,k,G,scm)
    # remove the current nb
    del H[dgr][score_dict[v]][v]
    # if there are no more nodes of this particular score, remove the dict
    if H[dgr][score_dict[v]] == {}:
        del H[dgr][score_dict[v]]
    # if the score was equal to the max score, update max_score_dict
    if score_dict[v] == max_score_dict[dgr]:
        try:
            max_score_dict[dgr] = max(H[dgr].keys())
        # no more nodes left of degree d
        except ValueError:
            max_score_dict[dgr] = None


In [3]:
def save_to_file(filepath,file_list,k=1):
    for name in file_list:
        print(name)        
        file = filepath+name#+".txt"
        fh = open(file, "rb")
        GRAPH = nx.read_edgelist(fh)
        fh.close()
        nodes = GRAPH.nodes()
        '''mapping = {}
        for i, j in enumerate(GRAPH):
            mapping[j] = i
        GRAPH = nx.relabel_nodes(GRAPH, mapping)'''
        #map = {n:int(i) for i, n in enumerate(nodes)}
        #GRAPH = nx.relabel_nodes(GRAPH, map)
        GRAPH.remove_edges_from(nx.selfloop_edges(GRAPH))
        D,N,removed_nodes = destroy(k,GRAPH)
        # open file in write mode
        with open(r'./COREHD_New/'+name, 'w') as fp:
            for item in D:
                # write each item on a new line
                fp.write("%s\n" % item)


In [ ]:
filepath = "../Dataset/Real/"
file_list =  ["test","testTrain","corruption","foodweb-baywet","inf-USAir97","moreno_crime_projected",'opsahl-openflights','household','faa','facebook','powergrid','netscience','HI-II-14']
file_list = [f+".txt" for f in file_list]
save_to_file(filepath,file_list,k=2)
'''file_list = ['ba_300_20_house_1', 'ba_300_40_house_2', 'ba_300_60_house_3', 'ba_300_80_house_4', 'ba_300_100_house_5', 'ba_300_20_grid_1', 'ba_300_40_grid_2', 'ba_300_60_grid_3', 'ba_300_80_grid_4', 'ba_300_100_grid_5']
filepath = "../Dataset/Validation/GNNexplanation/"
save_to_file(filepath,file_list,k=2)
file_list = [ 'tree_8_20_cycle_1', 'tree_8_40_cycle_2', 'tree_8_60_cycle_3', 'tree_8_80_cycle_4', 'tree_8_100_cycle_5', 'tree_8_20_grid_1', 'tree_8_40_grid_2', 'tree_8_60_grid_3', 'tree_8_80_grid_4', 'tree_8_100_grid_5']
filepath = "../Dataset/Validation/GNNexplanation/"
save_to_file(filepath,file_list,k=2)
file_list =['ba_60_10_house_1', 'ba_60_20_house_2', 'ba_60_30_house_3', 'ba_60_10_fan_1', 'ba_60_20_fan_2', 'ba_60_30_fan_3', 'ba_60_10_clique_1', 'ba_60_20_clique_2', 'ba_60_30_clique_3', 'ba_60_10_diamond_1', 'ba_60_20_diamond_2', 'ba_60_30_diamond_3', 'ba_60_10_star_1', 'ba_60_20_star_2', 'ba_60_30_star_3', 'ba_60_10_grid_1', 'ba_60_20_grid_2', 'ba_60_30_grid_3']
filepath = "../Dataset/Validation/GNNexplanation/New/"
save_to_file(filepath,file_list,k=2)
file_list = ['tree_4_10_house_1', 'tree_4_20_house_2', 'tree_4_30_house_3', 'tree_4_10_fan_1', 'tree_4_20_fan_2', 'tree_4_30_fan_3', 'tree_4_10_clique_1', 'tree_4_20_clique_2', 'tree_4_30_clique_3', 'tree_4_10_diamond_1', 'tree_4_20_diamond_2', 'tree_4_30_diamond_3', 'tree_4_10_cycle_1', 'tree_4_20_cycle_2', 'tree_4_30_cycle_3', 'tree_4_10_star_1', 'tree_4_20_star_2', 'tree_4_30_star_3', 'tree_4_10_grid_1', 'tree_4_20_grid_2', 'tree_4_30_grid_3']
filepath = "../Dataset/Validation/GNNexplanation/New/"
save_to_file(filepath,file_list,k=2)
filepath = "../Dataset/Validation/Motifs_Attached/Tree/"
file_list = ['tree_8_20_house_1', 'tree_8_40_house_2', 'tree_8_60_house_3', 'tree_8_80_house_4', 'tree_8_100_house_5', 'tree_8_20_fan_1', 'tree_8_40_fan_2', 'tree_8_60_fan_3', 'tree_8_80_fan_4', 'tree_8_100_fan_5', 'tree_8_20_clique_1', 'tree_8_40_clique_2', 'tree_8_60_clique_3', 'tree_8_80_clique_4', 'tree_8_100_clique_5', 'tree_8_20_diamond_1', 'tree_8_40_diamond_2', 'tree_8_60_diamond_3', 'tree_8_80_diamond_4', 'tree_8_100_diamond_5', 'tree_8_20_cycle_1', 'tree_8_40_cycle_2', 'tree_8_60_cycle_3', 'tree_8_80_cycle_4', 'tree_8_100_cycle_5', 'tree_8_20_star_1', 'tree_8_40_star_2', 'tree_8_60_star_3', 'tree_8_80_star_4', 'tree_8_100_star_5', 'tree_8_20_grid_1', 'tree_8_40_grid_2', 'tree_8_60_grid_3', 'tree_8_80_grid_4', 'tree_8_100_grid_5']
save_to_file(filepath,file_list,k=2)
filepath = "../Dataset/Validation/Motifs_Attached/BA/"
file_list = ['ba_300_20_house_1', 'ba_300_40_house_2', 'ba_300_60_house_3', 'ba_300_80_house_4', 'ba_300_100_house_5', 'ba_300_20_fan_1', 'ba_300_40_fan_2', 'ba_300_60_fan_3', 'ba_300_80_fan_4', 'ba_300_100_fan_5', 'ba_300_20_clique_1', 'ba_300_40_clique_2', 'ba_300_60_clique_3', 'ba_300_80_clique_4', 'ba_300_100_clique_5', 'ba_300_20_diamond_1', 'ba_300_40_diamond_2', 'ba_300_60_diamond_3', 'ba_300_80_diamond_4', 'ba_300_100_diamond_5', 'ba_300_20_cycle_1', 'ba_300_40_cycle_2', 'ba_300_60_cycle_3', 'ba_300_80_cycle_4', 'ba_300_100_cycle_5', 'ba_300_20_star_1', 'ba_300_40_star_2', 'ba_300_60_star_3', 'ba_300_80_star_4', 'ba_300_100_star_5', 'ba_300_20_grid_1', 'ba_300_40_grid_2', 'ba_300_60_grid_3', 'ba_300_80_grid_4', 'ba_300_100_grid_5']
save_to_file(filepath,file_list,k=2)'''

In [4]:
from os import listdir
from os.path import isfile, join
filepath = "../Dataset/SyntheticGraph/New/"
file_list =  [f for f in listdir(filepath) if isfile(join(filepath, f))]
save_to_file(filepath,file_list,k=2)

regular_250_4_1.txt
erdos_renyi_250_0.075_2.txt
barabasi_albert_250_20_3.txt
regular_250_8_4.txt
regular_250_10_5.txt
barabasi_albert_250_20_5.txt
erdos_renyi_250_0.1_3.txt
regular_250_14_5.txt
barabasi_albert_250_6_2.txt
regular_250_2_3.txt
barabasi_albert_250_2_3.txt
regular_250_4_2.txt
erdos_renyi_250_0.001_2.txt
erdos_renyi_250_0.075_5.txt
sbm_250_15_3.txt
small-world_250_20_0.15_1.txt
sbm_250_35_5.txt
erdos_renyi_250_0.3_1.txt
small-world_250_10_0.15_3.txt
sbm_250_40_3.txt
sbm_250_10_2.txt
erdos_renyi_250_0.03_3.txt
small-world_250_20_0.15_2.txt
regular_250_14_1.txt
regular_250_12_4.txt
barabasi_albert_250_12_4.txt
sbm_250_10_5.txt
small-world_250_8_0.15_2.txt
regular_250_14_3.txt
regular_250_8_3.txt
small-world_250_4_0.15_5.txt
sbm_250_45_4.txt
small-world_250_10_0.15_1.txt
regular_250_20_4.txt
sbm_250_40_2.txt
barabasi_albert_250_20_2.txt
small-world_250_2_0.15_4.txt
regular_250_6_5.txt
erdos_renyi_250_0.25_2.txt
barabasi_albert_250_16_4.txt
regular_250_6_2.txt
barabasi_albert_2

In [10]:
from os import listdir
from os.path import isfile, join
filepath = "../Dataset/SyntheticGraph/Homogeneity/New/"
file_list =  [f for f in listdir(filepath) if isfile(join(filepath, f))]
save_to_file(filepath,file_list,k=2)

erdos_renyi_100_0.050_4.txt
barabasi_albert_50_3.000_10.txt
barabasi_albert_350_3.000_3.txt
erdos_renyi_250_0.050_4.txt
small-world_100_4.000_1.txt
erdos_renyi_150_0.050_2.txt
barabasi_albert_150_3.000_9.txt
barabasi_albert_200_3.000_6.txt
barabasi_albert_400_3.000_9.txt
barabasi_albert_100_3.000_2.txt
small-world_450_4.000_4.txt
erdos_renyi_50_0.050_3.txt
erdos_renyi_450_0.050_10.txt
barabasi_albert_100_3.000_10.txt
erdos_renyi_150_0.050_5.txt
small-world_50_4.000_9.txt
small-world_350_4.000_1.txt
erdos_renyi_350_0.050_3.txt
barabasi_albert_250_3.000_1.txt
erdos_renyi_500_0.050_8.txt
erdos_renyi_400_0.050_7.txt
small-world_350_4.000_8.txt
barabasi_albert_500_3.000_6.txt
small-world_400_4.000_2.txt
small-world_450_4.000_9.txt
barabasi_albert_500_3.000_10.txt
erdos_renyi_400_0.050_8.txt
erdos_renyi_350_0.050_8.txt
erdos_renyi_200_0.050_3.txt
erdos_renyi_50_0.050_8.txt
small-world_250_4.000_7.txt
barabasi_albert_100_3.000_6.txt
small-world_350_4.000_7.txt
erdos_renyi_500_0.050_5.txt
bara

small-world_500_4.000_1.txt
erdos_renyi_350_0.050_10.txt
erdos_renyi_250_0.050_5.txt
erdos_renyi_400_0.050_10.txt
small-world_400_4.000_5.txt
barabasi_albert_200_3.000_1.txt
small-world_450_4.000_2.txt
erdos_renyi_250_0.050_10.txt
barabasi_albert_350_3.000_6.txt
erdos_renyi_350_0.050_9.txt
barabasi_albert_350_3.000_10.txt
erdos_renyi_150_0.050_9.txt
erdos_renyi_450_0.050_8.txt
erdos_renyi_150_0.050_6.txt
small-world_450_4.000_3.txt
barabasi_albert_350_3.000_7.txt
small-world_250_4.000_4.txt
erdos_renyi_450_0.050_5.txt


In [ ]:
from os import listdir
from os.path import isfile, join
filepath = "../Dataset/Motifs_Attached/New_BA/"
file_list =  [f for f in listdir(filepath) if isfile(join(filepath, f))]
save_to_file(filepath,file_list,k=2)

In [ ]:
from os import listdir
from os.path import isfile, join
filepath = "../Dataset/Motifs_Attached/New_Tree/"
file_list =  [f for f in listdir(filepath) if isfile(join(filepath, f))]
save_to_file(filepath,file_list,k=2)

In [ ]:
filepath = "../Dataset/GeneNetwork/"
cancer=  ["HNSC","KICH","KIRP","STAD","UCEC"]
file_list = []
for c in cancer:
    for i in range(1,6):
        file_list.append(c+"_"+str(i)+".txt")
save_to_file(filepath,file_list,k=2)

In [ ]:
filepath = "../Dataset/GeneNetwork/Pair-SSN/"
cancer=  ["KICH","KIRP","STAD"]
file_list = []
for c in cancer:
    for i in range(1,11):
        file_list.append(c+"_"+str(i)+".txt")
save_to_file(filepath,file_list,k=2)

In [ ]:
filepath = "../Dataset/GridNetwork/"
file_list = ["gridkit_europe.txt","gridkit_north_america.txt"]
save_to_file(filepath,file_list,k=2)